## Imports and Setup

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from   sklearn.decomposition import TruncatedSVD
from   sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
# class corpus metadata
metadata = pd.read_csv("class_corpus_metadata.csv")

metadata.shape

(160, 34)

In [5]:
# training data are books that are either horror or detective
training_data = metadata[(metadata['horror']==True) | (metadata['detective']==True)]

# drop books that are both horror and detective
drop = metadata[(metadata['horror']==True) & (metadata['detective']==True)]
training_data = training_data.drop(drop.index)

# testing data are books are neither horror or detective
testing_data = metadata[(metadata['horror']==False) & (metadata['detective']==False)]

# sort titles alphabetically 
training_data = training_data.sort_values('title')
testing_data = testing_data.sort_values('title')
# note: training+testing+dropped row = 159 rows, class corpus = 160 rows, "An Unkindness of Ghosts" has no input for horror and detective column

In [6]:
training_names = training_data.filename.values
testing_names = testing_data.filename.values

In [9]:
training_names[0]

'Leblanc_813.txt'

In [11]:
training_books=[]
for book in training_names:
    with open(book, 'r',encoding='utf-8') as f:
        file = f.read()
        training_books.append(file)
        

FileNotFoundError: [Errno 2] No such file or directory: 'Wharton_EthanFrome.txt'

In [12]:
training_books

['\nCHAPTER I\n\nTHE TRAGEDY AT THE PALACE HOTEL\n\n\nMr. Kesselbach stopped short on the threshold of the sitting-room, took\nhis secretary\'s arm and, in an anxious voice, whispered:\n\n"Chapman, some one has been here again."\n\n"Surely not, sir," protested the secretary. "You have just opened the\nhall-door yourself; and the key never left your pocket while we were\nlunching in the restaurant."\n\n"Chapman, some one has been here again," Mr. Kesselbach repeated. He\npointed to a traveling-bag on the mantelpiece. "Look, I can prove it.\nThat bag was shut. It is now open."\n\nChapman protested.\n\n"Are you quite sure that you shut it, sir? Besides, the bag contains\nnothing but odds and ends of no value, articles of dress. . . ."\n\n"It contains nothing else, because I took my pocket-book out before we\nwent down, by way of precaution. . . . But for that. . . . No, Chapman,\nI tell you, some one has been here while we were at lunch."\n\nThere was a telephone on the wall. He took down

In [ ]:
with open 